In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
layers = keras.layers

In [20]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Code for calculating LE

In [18]:
def rnn_jac(Wxh, Whh, ht, xt, phiprime):
    """
    Compute the Jacobian of the RNN with respect to the hidden state ht
    :param Wxh: input-to-hidden weight matrix (U)
    :param Whh: hidden-to-hidden weight matrix (V)
    :param ht: current hidden state
    :param xt: current input
    :param phiprime: function handle for the derivative of the activation function
    :return: Jacobian matrix
    """
    ht = np.reshape(ht, [-1, 1])  # shape: (32, 1)
    xt = np.reshape(xt, [-1, 1])  # shape: (32, 1)
    # Compute the Jacobian of the RNN with respect to ht


    alpha=Wxh@xt + Whh@ht
    J=np.diag(phiprime(alpha).flatten())@Whh
    return J

def calc_LEs(x_batches, h0, RNNlayer, activation_function_prim=lambda x:np.heaviside(x,1), k_LE=1000):
    """
    Calculate the Lyapunov exponents of a batch of sequences using the QR method.
    :param x_batches: input sequences (batch_size, T, input_size)
    :param h0: initial hidden state (batch_size, hidden_size)
    :param RNNlayer: RNN layer object (e.g., tf.keras.layers.SimpleRNN)
    :param activation_function_prim: function handle to derivative of activation function used in the RNN layer
    :param k_LE: number of Lyapunov exponents to compute
    :return: Lyapunov exponents for each batch (batch_size, k_LE)
    """
    #get dimensions
    batch_size, hidden_size = h0.shape
    batch_sizeX, T, input_size = x_batches.shape
    if batch_size != batch_sizeX:
        raise ValueError("batch size of h and X not compatible")
    L = hidden_size

    #get recurrent cell
    RNNcell=RNNlayer.cell

    # Choose how many exponents to track
    k_LE = max(min(L, k_LE), 1)

    #save average Lyapunov exponent over the sequence for each batch
    lyaps_batches = np.zeros((batch_size, k_LE))
    #Loop over input sequence
    for batch in range(batch_size):
        x=x_batches[batch]
        ht=h0[batch]
        #Initialize Q
        Q = tf.eye(L)
        #keep track of average lyapunov exponents
        cum_lyaps = tf.zeros((k_LE,))

        for t in range(T):
            #Get next state ht+1 by taking a reccurent step
            xt=x[t]
            xt = tf.cast(tf.reshape(xt, [1, input_size]), tf.float32); ht = tf.cast(tf.reshape(ht, [1, L]), tf.float32); _, ht = RNNcell(xt, ht)

            #Get jacobian J
            Wxh, Whh, b = RNNlayer.get_weights()
            # Transpose to match math-style dimensions
            Wxh = Wxh.T  # Now shape (units, input_dim)
            Whh = Whh.T  # Now shape (units, units)
            J = rnn_jac(Wxh, Whh, ht, xt, activation_function_prim)
            #Get the Lyapunov exponents from qr decomposition
            Q=Q@J
            Q,R=tf.linalg.qr(Q, full_matrices=False)
            cum_lyaps += tf.math.log(tf.math.abs(tf.linalg.diag_part(R[0:k_LE, 0:k_LE])))
        lyaps_batches[batch] = cum_lyaps / T
    return lyaps_batches


# Code used to test/show implementation

Start out with defining and training a toy model

In [3]:
def define_model():
    """Define and compile a simple RNN model."""
    z0 = layers.Input(shape=[None, 2])  # time steps unspecified, 2 features
    z = layers.SimpleRNN(32, activation="tanh")(z0)
    z = layers.Dense(32, activation='relu')(z)
    z = layers.Dense(16, activation='relu')(z)
    z = layers.Dense(1)(z)

    model = keras.models.Model(inputs=z0, outputs=z)
    model.compile(loss='mse', optimizer='adam')
    return model

def train_model(model, X, y, epochs=20, batch_size=10):
    """Train model with early stopping and LR scheduler."""
    results = model.fit(
        X, y,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        verbose=1,
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(factor=0.67, patience=3, verbose=1, min_lr=1E-5),
            keras.callbacks.EarlyStopping(patience=4, verbose=1)
        ]
    )
    return results

# Create some toy data
n_samples = 300
time_steps = 20
X = np.random.rand(n_samples, time_steps, 2)  # [batch, time, features]
Y = np.random.rand(n_samples)

# Create and train model
model = define_model()
results = train_model(model, X, Y)

Epoch 1/20
27/27 [==============================] - 2s 22ms/step - loss: 0.2036 - val_loss: 0.0852 - lr: 0.0010
Epoch 2/20
27/27 [==============================] - 0s 15ms/step - loss: 0.0942 - val_loss: 0.0857 - lr: 0.0010
Epoch 3/20
27/27 [==============================] - 0s 15ms/step - loss: 0.0937 - val_loss: 0.0863 - lr: 0.0010
Epoch 4/20
27/27 [==============================] - 0s 15ms/step - loss: 0.0896 - val_loss: 0.0847 - lr: 0.0010
Epoch 5/20
27/27 [==============================] - 0s 16ms/step - loss: 0.0883 - val_loss: 0.0838 - lr: 0.0010
Epoch 6/20
27/27 [==============================] - 0s 15ms/step - loss: 0.0874 - val_loss: 0.0824 - lr: 0.0010
Epoch 7/20
27/27 [==============================] - 0s 15ms/step - loss: 0.0900 - val_loss: 0.0798 - lr: 0.0010
Epoch 8/20
27/27 [==============================] - 0s 15ms/step - loss: 0.0869 - val_loss: 0.0814 - lr: 0.0010
Epoch 9/20
27/27 [==============================] - 0s 15ms/step - loss: 0.0879 - val_loss: 0.0835 - lr:

Now we can calulate the LEs of the model

In [19]:
#create some batches of input data and initial hidden states
batch_size = 10   # number of sequences
T = 20            # length of each sequence
input_dim = 2     # size of each x
hidden_dim = 32   # size of hidden state

X = np.random.rand(batch_size, T, input_dim)
H0 = np.random.rand(batch_size, hidden_dim)

#Get the rnn layer of the model
rnn_layer=model.layers[1]

#Define the derivative of the activation function used
tanh_prim=lambda x: 1-np.power(np.tanh(x), 2)

#calculate the LEs
number_exponents=20
LEs=calc_LEs(X,H0, rnn_layer, tanh_prim, number_exponents)
#LEs[batch, exponent], exponents are not ordered
print(LEs)

[[-0.08115374 -0.13219818 -0.18581873 -0.12058721 -0.18589953 -0.13813955
  -0.09557717 -0.17561284 -0.18069313 -0.20732288 -0.11960275 -0.14068384
  -0.16383703 -0.13621159 -0.20968314 -0.1637935  -0.1433326  -0.15949938
  -0.17264342 -0.19374862]
 [-0.08273657 -0.16165161 -0.2133095  -0.11809427 -0.19154061 -0.16152157
  -0.12143986 -0.22078271 -0.20351167 -0.23067811 -0.14552125 -0.18154164
  -0.18714103 -0.16781119 -0.21152203 -0.18963881 -0.13863254 -0.16378766
  -0.16923454 -0.22941992]
 [-0.08641966 -0.1543588  -0.23745427 -0.12475763 -0.20036158 -0.17288859
  -0.11694592 -0.21285319 -0.21934381 -0.25415507 -0.14834574 -0.18094803
  -0.19039956 -0.17788833 -0.21843679 -0.18241246 -0.15866964 -0.17567238
  -0.18474451 -0.23269439]
 [-0.0953928  -0.17563416 -0.24302678 -0.12851161 -0.20398378 -0.18718439
  -0.11926951 -0.20912775 -0.22246298 -0.26288754 -0.15892358 -0.19757919
  -0.20187104 -0.18665013 -0.21957636 -0.19645441 -0.15737028 -0.17545792
  -0.19852373 -0.23250768]
 [-0